# Python Practice Lecture 11 MATH 342W Queens College - Multivariate Linear Regression with the Hat Matrix
## Author: Amir ElTabakh
## Date: March 8, 2021

## Agenda:
* QR Decomposition

## QR Decomposition

Let's load the Boston Housing data again and regenerate all our quantities:

In [1]:
# Lines below are just to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Importing dependencies
from sklearn import datasets
import pandas as pd
import numpy as np

# Load the Boston Housing dataset as bh
bh = datasets.load_boston()

# Initialize target variable
y = bh.target

# Create Boston Housing df
X = pd.DataFrame(data = bh.data, columns = bh.feature_names)

# add intercept column
X.insert(0, 'INTERCEPT', [1 for i in range(len(X))])

# convert X to matrix object not dataframe
X.to_numpy()

# Load the first 5 rows of df
X.head()

,INTERCEPT,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,1,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,1,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,1,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [2]:
# SST
ybar = np.mean(y)

sst = sum((y - ybar)**2)
sst

42716.2954150198

In [3]:
n = len(X)
p_plus_one = len(X.iloc[0, :])

In [4]:
# solving for b
Xt = X.transpose()
XtXinv = np.linalg.inv(Xt @ X)
b = (XtXinv @ Xt @ y).to_numpy() # np array object is easier to do matrix multi with
b

array([ 3.64594884e+01, -1.08011358e-01,  4.64204584e-02,  2.05586264e-02,
        2.68673382e+00, -1.77666112e+01,  3.80986521e+00,  6.92224640e-04,
       -1.47556685e+00,  3.06049479e-01, -1.23345939e-02, -9.52747232e-01,
        9.31168327e-03, -5.24758378e-01])

In [5]:
# solving for yhat
yhat = X.dot(b)
yhat

0      30.003843
1      25.025562
2      30.567597
3      28.607036
4      27.943524
         ...    
501    23.533341
502    22.375719
503    27.627426
504    26.127967
505    22.344212
Length: 506, dtype: float64

In [6]:
# SSR (sum of squared residuals)
ssr = sum((yhat - ybar)**2)
ssr

31637.510837065387

In [7]:
# R^2
Rsq = ssr / sst
Rsq

0.7406426641094228

Now let's do the QR decomposition and see if the projections work. We'll use NumPy's `np.linalg.qr()` method.

In [8]:
print(n)
print(p_plus_one)

506
14


In [9]:
Q, R = np.linalg.qr(X)

# output dimension of Q, R
print(Q.shape)
print(R.shape)

(506, 14)
(14, 14)


In [10]:
# output rank of Q, R
print(np.linalg.matrix_rank(Q))
print(np.linalg.matrix_rank(R))

14
14


In [11]:
# snapshot of Q
Q[0:5, 0:5]

array([[-0.04445542, -0.01866158,  0.00910601, -0.05766684, -0.00830254],
       [-0.04445542, -0.01855299, -0.02592754, -0.03907578, -0.01166524],
       [-0.04445542, -0.0185531 , -0.02592756, -0.03907574, -0.01166525],
       [-0.04445542, -0.01852682, -0.02592218, -0.07931947, -0.00856873],
       [-0.04445542, -0.01833705, -0.02588335, -0.07939459, -0.00855013]])

In [12]:
# snapshot of R
R[0:5, 0:5]

array([[-2.24944438e+01, -8.12842024e+01, -2.55618679e+02,
        -2.50515641e+02, -1.55593979e+00],
       [ 0.00000000e+00,  1.93295685e+02, -1.05067310e+02,
         6.26818497e+01, -3.19018318e-01],
       [ 0.00000000e+00,  0.00000000e+00,  5.13467576e+02,
        -7.11779587e+01, -3.14032817e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.21541052e+02,  4.36290368e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  5.67347449e+00]])

In [13]:
1 / np.sqrt(n)

0.044455422447438706

In [14]:
# Is Q Normalized?
print(sum(Q[:, 1]**2))
print(sum(Q[:, 2]**2))

0.9999999999999997
1.0000000000000004


In [15]:
# Is Q Orthogonal?
print(Q[:, 1] @ Q[:, 2])
print(Q[:, 7] @ Q[:, 13])

-1.734723475976807e-17
2.3418766925686896e-17


In [16]:
# Get yhat via Q
yhat_via_Q = Q @ Q.transpose() @ y

# output yhats
print(yhat[0:5])
print(yhat_via_Q[0:5])

0    30.003843
1    25.025562
2    30.567597
3    28.607036
4    27.943524
dtype: float64
[30.00384338 25.02556238 30.56759672 28.60703649 27.94352423]


Can we get the $b$ vector from the $Q$ matrix?

In [17]:
np.linalg.inv(R) @ Q.transpose() @ y

array([ 3.64594884e+01, -1.08011358e-01,  4.64204584e-02,  2.05586264e-02,
        2.68673382e+00, -1.77666112e+01,  3.80986521e+00,  6.92224640e-04,
       -1.47556685e+00,  3.06049479e-01, -1.23345939e-02, -9.52747232e-01,
        9.31168327e-03, -5.24758378e-01])

In [18]:
b_Q = Q.transpose() @ y
b_Q

array([-506.86294458,  -80.25448934,   59.61825389,  -50.50976574,
         39.11326989,   -8.73198992,  104.58545124,    9.50094264,
         42.18413341,   -5.84245863,  -18.15362413,  -36.18437976,
        -24.35852139,  -49.10029215])

In [19]:
Q @ b_Q

array([30.00384338, 25.02556238, 30.56759672, 28.60703649, 27.94352423,
       25.25628446, 23.00180827, 19.53598843, 11.52363685, 18.92026211,
       18.99949651, 21.58679568, 20.90652153, 19.55290281, 19.28348205,
       19.29748321, 20.52750979, 16.91140135, 16.17801106, 18.40613603,
       12.52385753, 17.67103669, 15.83288129, 13.80628535, 15.67833832,
       13.38668561, 15.46397655, 14.70847428, 19.54737285, 20.8764282 ,
       11.45511759, 18.05923295,  8.81105736, 14.28275814, 13.70675891,
       23.81463526, 22.34193708, 23.10891142, 22.91502612, 31.35762569,
       34.21510225, 28.02056414, 25.20386628, 24.60979273, 22.94149176,
       22.09669817, 20.42320032, 18.03655088,  9.10655377, 17.20607751,
       21.28152535, 23.97222285, 27.6558508 , 24.04901809, 15.3618477 ,
       31.15264947, 24.85686978, 33.10919806, 21.77537987, 21.08493555,
       17.8725804 , 18.51110208, 23.98742856, 22.55408869, 23.37308644,
       30.36148358, 25.53056512, 21.11338564, 17.42153786, 20.78

In [20]:
X @ b

0      30.003843
1      25.025562
2      30.567597
3      28.607036
4      27.943524
         ...    
501    23.533341
502    22.375719
503    27.627426
504    26.127967
505    22.344212
Length: 506, dtype: float64

Nope - this is not the same! Why not?

Each dimension gives one piece of SSR and thus one piece of R^2 i.e. SSR = SSR_1 + ... + SSR_p and R^2 = R^2_1 + ... + R^2_p

Our definition of SSR removed the ybar i.e. the contribution of the intercept. So we will do so here. That is the first column of $Q$. Now we add up all the features besides the intercept.

In [21]:
# SSR list (you can populate an empty list)
partial_SSRs = []

for j in range(1, p_plus_one):
    # get row at column j
    qj = Q[:, j]
    
    # calculate yhat_j - we'll use the `.dot()` method for matrix multiplication
    yhat_j = qj.dot(qj.transpose().dot(y))
    
    # casting to int to round
    SSR_j = sum(yhat_j**2)
    
    # add value to end of list
    partial_SSRs += [SSR_j]
    
print(sum(partial_SSRs))
print(ssr)

31637.510837064816
31637.510837065387


In [22]:
# output SST
print(sst)

42716.2954150198


In [23]:
# Rsq
partial_rsqs = partial_SSRs / sst
sum(partial_rsqs)
print(partial_rsqs)

[0.15078047 0.08320797 0.05972513 0.03581415 0.00178498 0.25606426
 0.0021132  0.0416586  0.00079909 0.00771495 0.03065128 0.01389019
 0.05643838]


Some dimensions in this subspace matter more than others. We can do approximately the same regression with less than p features. Let's try this:

In [24]:
# sort
partial_rsqs_sorted = np.sort(partial_rsqs)[::-1] # descending
print(partial_rsqs_sorted)

[0.25606426 0.15078047 0.08320797 0.05972513 0.05643838 0.0416586
 0.03581415 0.03065128 0.01389019 0.00771495 0.0021132  0.00178498
 0.00079909]


In [25]:
# get indices of sort
partial_rsqs_sorted_indices = np.argsort(partial_rsqs)[::-1]
partial_rsqs_sorted_indices

array([ 5,  0,  1,  2, 12,  7,  3, 10, 11,  9,  6,  4,  8], dtype=int64)

In [26]:
# cumulative sum
partial_Rsqs_sorted_cumul = np.cumsum(partial_rsqs_sorted)
print(partial_Rsqs_sorted_cumul)

[0.25606426 0.40684473 0.4900527  0.54977783 0.60621622 0.64787482
 0.68368897 0.71434025 0.72823045 0.7359454  0.73805859 0.73984357
 0.74064266]


In [27]:
# sort Q by Rsq
Q_sorted = [Q[:, i] for i in partial_rsqs_sorted_indices]
Q_sorted

[array([ 5.55571245e-02, -2.69290576e-02, -2.69290341e-02, -1.93513585e-03,
        -1.97831435e-03, -1.93216939e-03,  7.93767519e-03,  7.87144774e-03,
         7.79294245e-03,  7.84144175e-03,  7.77687411e-03,  7.90332544e-03,
         7.93121254e-03,  8.26168307e-03,  8.25203030e-03,  8.26447296e-03,
         7.76236394e-03,  8.07988132e-03,  8.05809195e-03,  8.14862790e-03,
         7.52944958e-03,  8.00002228e-03,  7.55219246e-03,  7.83946841e-03,
         8.11983439e-03,  8.01355971e-03,  8.21206546e-03,  7.87791470e-03,
         8.09307737e-03,  7.82296452e-03,  7.67186330e-03,  7.40828374e-03,
         7.36911937e-03,  7.64724873e-03,  7.10445681e-03, -1.10924360e-03,
        -1.14840796e-03, -1.12804297e-03, -1.23976793e-03,  1.02902246e-02,
         1.02832087e-02, -3.90513742e-02, -3.90679252e-02, -3.90889494e-02,
        -3.90457827e-02, -3.91031461e-02, -3.91230873e-02, -3.91712452e-02,
        -3.92002035e-02, -3.91600621e-02, -2.62858722e-02, -2.62324759e-02,
        -2.6

In [28]:
Q_reduced = pd.DataFrame(Q_sorted).transpose() # DataFrame's can be transposed
Q_reduced

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.055557,-0.044455,-0.018662,0.009106,-0.004993,-0.013978,-0.057667,0.055974,-0.038282,-0.030550,-0.001676,-0.008303,0.019657
1,-0.026929,-0.044455,-0.018553,-0.025928,-0.001713,-0.049516,-0.039076,-0.015651,-0.021045,-0.021991,0.005421,-0.011665,-0.039188
2,-0.026929,-0.044455,-0.018553,-0.025928,-0.001465,-0.001655,-0.039076,-0.008740,-0.003960,-0.028975,0.059051,-0.011665,-0.037163
3,-0.001935,-0.044455,-0.018527,-0.025922,-0.000271,0.025673,-0.079319,-0.008083,0.021435,-0.008671,0.034898,-0.008569,-0.054818
4,-0.001978,-0.044455,-0.018337,-0.025883,-0.001609,0.005504,-0.079395,-0.007774,0.021662,-0.008885,0.045372,-0.008550,-0.063453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.006000,-0.044455,-0.018370,-0.025890,-0.003935,0.013152,0.000838,-0.007627,0.102378,-0.059726,0.026903,-0.014722,0.033675
502,0.006021,-0.044455,-0.018460,-0.025909,-0.005285,-0.007982,0.000874,-0.011393,0.093455,-0.055316,-0.006307,-0.014731,0.043001
503,0.006002,-0.044455,-0.018380,-0.025892,-0.006384,-0.039500,0.000842,-0.006519,0.104107,-0.059280,0.053788,-0.014723,0.024283
504,0.005945,-0.044455,-0.018127,-0.025840,-0.004464,-0.036109,0.000742,-0.007889,0.100717,-0.058872,0.041031,-0.014698,0.019041


In [29]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(Q_reduced, y)

model.score(Q_reduced, y)

0.68420427096417